In [292]:
import numpy as np

In [293]:
def roll_condition(wheel_data): 
    
    roll_matrix = np.zeros((len(wheel_data), 3))
    
    for i in range(len(wheel_data)):
        wheel_type = wheel_data[i+1]['wheel_type']
        alpha = wheel_data[i+1]['alpha'] * np.pi/180
        beta = wheel_data[i+1]['beta'] * np.pi/180
        gamma = wheel_data[i+1]['gamma'] * np.pi/180
        l = wheel_data[i+1]['l']
        d = wheel_data[i+1]['d']
        r = wheel_data[i+1]['r']

        roll_matrix[i,:] = np.array([np.sin(alpha+beta+gamma), -np.cos(alpha+beta+gamma), -l*np.cos(beta+gamma)])
    
    
    return np.around(roll_matrix,4)

In [294]:
def radius_data(wheel_data):
    
    a=np.array([])
    for i in range(len(wheel_data)):
        a = np.append(a, wheel_data[i+1]['r'] * np.cos(wheel_data[i+1]['gamma'] * np.pi/180))   
        
    return np.around(np.diag(a),4)

In [295]:
def no_slip_condition(wheel_data):
    
    no_slip_matrix = np.empty((1,3))
    dos_matrix = np.empty((1,3))

    for i in range(len(wheel_data)):
        wheel_type = wheel_data[i+1]['wheel_type']
        alpha = wheel_data[i+1]['alpha'] * np.pi/180
        beta = wheel_data[i+1]['beta'] * np.pi/180
        gamma = wheel_data[i+1]['gamma'] * np.pi/180
        l = wheel_data[i+1]['l']
        d = wheel_data[i+1]['d']
        r = wheel_data[i+1]['r']
        
        if (wheel_type == 'f' or wheel_type == 's'):
            temp_array = np.array([[np.cos(alpha+beta+gamma), np.sin(alpha+beta+gamma), l*np.sin(beta+gamma)]])
            if wheel_type == 's':
                dos_matrix = np.vstack((dos_matrix, temp_array))
            no_slip_matrix = np.vstack((no_slip_matrix, temp_array))
        temp_array = np.around(no_slip_matrix[1:,:],4)
        
        #Degree of Mobility and Steerability
        dos = np.linalg.matrix_rank(dos_matrix[1:,:]) if dos_matrix[1:,:].size != 0 else 0
        dom = 3-np.linalg.matrix_rank(temp_array)
    return temp_array, (dom, dos)

In [299]:
#Robot Wheel Definition
wheel_data = {1: {'wheel_type':'f' , 'alpha':-90, 'beta':0, 'gamma':0, 'l':250, 'd':0, 'r':100},
             2: {'wheel_type':'f' , 'alpha':90, 'beta':0, 'gamma':0, 'l':250, 'd':0, 'r':100},
             3: {'wheel_type':'sw' , 'alpha':0, 'beta':0, 'gamma':45, 'l':250, 'd':0, 'r':100},
             4: {'wheel_type':'sw' , 'alpha':120, 'beta':0, 'gamma':45, 'l':250, 'd':0, 'r':100},
             5: {'wheel_type':'sw' , 'alpha':-120, 'beta':0, 'gamma':45, 'l':250, 'd':0, 'r':100}}

In [297]:
J1 = roll_condition(wheel_data)
J2 = radius_data(wheel_data)
C1, (dom,dos) = no_slip_condition(wheel_data)

In [306]:
#Check if velocity is possible or not

v = n=list(map(float, input('Enter Velocity [x, y, theta]: ').strip().split(',')))
print(v, 'm/s')

v = np.reshape(np.array(v), (-1,1))
check = 'Velocity Not Possible' if (C1 @ v).all() != 0 else 'Velocity Possible'
print(check)

Enter Velocity [x, y, theta]:  1,2,3


[1.0, 2.0, 3.0] m/s
Velocity Not Possible
